# Ejercicio práctico - Tema 2: Modelo de Espacio Vectorial

**Alumno**: Iván Cañaveral Sánchez

El objetivo de este ejercicio práctico es el estudio de la representación dentro del
modelo de espacio vectorial. Para ello vamos a generar representaciones dentro de este
modelo con diferentes funciones de pesado y evaluarlas dentro de un problema de
clustering de documentos.

Como objetivo principal, vamos a generar represeantaciones vectoriales empleando estas tres funciones de pesado:
* TF
* TF-IDF
* Binary

Una vez obtenidos los vectores de representación se ejecutará un algoritmo de
clustering y se compararán los resultados obtenidos, tratándolos de relacionar con la
función de pesado aplicada en cada caso.

A continuación analizamos IDF en este problema,
calculando la frecuencia de documentos de cada término, y ver si existen
términos que caractericen algún grupo o si en caso contrario, la penalización del factor IDF a aquellos
términos presentes en un número elevado (o no tan elevado) de documentos es una
estimación correcta para esta tarea.

Tras esto, evaluaremos el impacto que tiene la aplicación de LSI en este problema.

## Librerías y constantes

En este apartado importamos las librerías necesarias para el desarrollo del ejercicio práctico. También fijamos algunos parámetros de la vectorización com variables globales, para que queden fijadas apra todas las representaciones.

Las principales librerías que utilizaremos serán:
* `numpy`
* `pandas`
* `seaborn`
* `sklearn`

Como algoritmo de clústering, elegiremos `KMeans` por su simplicidad.


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns
sns.set_theme()
sns.set(rc={'figure.figsize':(11,8)})
sns.color_palette("crest", as_cmap=True)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

In [ ]:
from collections import defaultdict, Counter
from sklearn import metrics
from time import time

In [ ]:
MAX_DF = 1.0
MIN_DF = 10
MAX_FEATURES = 10_000
SEED = 73

## Carga del detaset


Vamos  acargar un dataset público conocido como `20newsgroups_dataset`, que consiste en unos 18.000 textos repartidos en varias categorías. Elegimos este conjunto de datos dadas las facilidades que las principales librerías de ML ofrecen para su descarga. Está incluído en las principales librerías, entre ellas `sklearn` o `tensorflow`.

Para acotar el ámbito de la práctica, vamos e seleccionar las siguientes categorías:

```
    "alt.atheism",
    "talk.religion.misc",
    "comp.graphics",
    "sci.space",
    "rec.autos",
    "rec.motorcycles"
```

En el dataset podemos encontrar otros tipos de meta información como `"headers"`, `"footers"` o `"quotes"` que por el momento vamos a descartar.


In [ ]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups

categories = [
    "alt.atheism",
    "talk.religion.misc",
    "comp.graphics",
    "sci.space",
    "rec.autos",
    "rec.motorcycles"
]

dataset = fetch_20newsgroups(
    remove=("headers", "footers", "quotes"),
    subset="all",
    categories=categories,
    shuffle=True,
    random_state=42,
)

labels = dataset.target
unique_labels, category_sizes = np.unique(labels, return_counts=True)
true_k = unique_labels.shape[0]

print(f"{len(dataset.data)} documents - {true_k} categories")

Vamos a explorar brevemente la información del conjunto de datos. Aquí tenemos uno de los textos del dataset:

In [ ]:
dataset["data"][0]

Y a continuación podemos ver las etiquetas de los datasets:

In [ ]:
dataset["target"][:10]

## Vectorizadores

Vamos a crear a continuación los objetos que vamos a utilizar para generar las representaciones. En todos ellos fijaremos los parámetros `MAX_DF`, `MIN_DF` y `MAX_FEATURES`.

Vamos a apoyarnos en las clases `CountVectorizer` y `TfidfVectorizer`, que permiten generar las representaciones esperadas.

In [ ]:
# con el parámetro binary=False, la representación ofrecerá el número de 
# veces que aparece una palabra en un texto
tf_vectorizer = CountVectorizer(
    max_df=MAX_DF,
    min_df=MIN_DF,
    max_features=MAX_FEATURES,
    binary=False,
)

In [ ]:
tf_idf_vectorizer = TfidfVectorizer(
    max_df=MAX_DF,
    min_df=MIN_DF,
    max_features=MAX_FEATURES,
    use_idf=True, # aplicamos idf
    smooth_idf=1
)

In [ ]:
# con el parámetro binary=True, la representación ofrecerá variables 
# binarias que indican si aparece una palabra en un texto
binary_vectorizer = CountVectorizer(
    max_df=MAX_DF,
    min_df=MIN_DF,
    max_features=MAX_FEATURES,
    binary=True
)

### Ajuste

Ahora vamos a ajustar estos vectorizadores y a generar las representaciones.

Vamos a generar un diccionario para guardar estos objetos (`vectorizers`), y otro para las representaciones (`vectors`).

In [ ]:
vectorizers = {
    'tf': tf_vectorizer,
    'tf_idf': tf_idf_vectorizer,
    'binary': binary_vectorizer
}

In [ ]:
vectors = {}

In [ ]:
for vectorizer_name, vectorizer in vectorizers.items():
  print(f'fitting {vectorizer_name} vectorizer ...')
  vectors[vectorizer_name] = vectorizer.fit_transform(dataset.data)

Comprobamos cuántos textos y variables se han registrado en cada representación:

In [ ]:
for vectorizer_name, X in vectors.items():
  print(f"{vectorizer_name}: n_samples: {X.shape[0]}, n_features: {X.shape[1]}")

Las matrices de estas representaciones sulen ser poco densas. Para intentar metrizar esto de alguna manera, vamos a intentar medir el porcentaje de elementos que guardan estas matrices:

In [ ]:
for vectorizer_name, X in vectors.items():
  print(f"{vectorizer_name}: {X.nnz / np.prod(X.shape):.3f}")

Finalmente vamos a revisar las representaciones generadas, entendiendo cómo son los valores distintos de cero. Observamos que en el caso de `tf` son enteros, `binary` valores binarios, y `tf_idf` son números no enteros, com cabía esperar.

In [ ]:
vectors['tf'][vectors['tf'].nonzero()]

In [ ]:
vectors['binary'][vectors['binary'].nonzero()]

In [ ]:
vectors['tf_idf'][vectors['tf_idf'].nonzero()]

### Cálculo de valores idf

Dado que uno de los elementos que revisaremos en esta práctica son los valores de idf, vamos a calcularlos y a explorarlos brevemente para ganar intuición sobre la pregunta a responder en esta práctica sobre su impacto.

In [ ]:
idf_values = {k:v for k, v in zip(vectorizers['tf_idf'].get_feature_names_out(), vectorizers['tf_idf'].idf_)}

Revisemos a continuación los elementos con menor valor de idf:

In [ ]:
sorted_idf_values = dict(sorted(idf_values.items(), key=lambda item: item[1], reverse=False))
for word, value in list(sorted_idf_values.items())[:10]:
  print(f"Word: {word} \t Value: {value}")

Como podemos observar, las palabras con menos valor son principalmente palabras de baja carga semántica (stopwords en su mayoría).

Revisemos ahora con mayores valores de `idf`:

In [ ]:
sorted_idf_values = dict(sorted(idf_values.items(), key=lambda item: item[1], reverse=True))
for word, value in list(sorted_idf_values.items())[50:60]:
  print(f"Word: {word} \t Value: {value}")

Muchas de ellas parecen palabras con una mayor carga semántica, y que pueden tener un mayor peso en la generación del clústering.

Revisaremos su impacto en las próximas secciones.

## Algoritmo clustering

Como algoritmo de clústering hemos elegido uno de los más sencillos, por simplicidad y para poder centrarnos en la representación.

Sin embargo, debemos tener en cuenta un punto que, si bien no es específico de `KMeans`, debemos tener en mente para poder interpretar correctamente los resultados.

Cuando trabajamos con dimensionalidades altas `k-means` puede inicializar centroides en puntos de datos extremadamente aislados. Esos puntos de datos pueden seguir siendo sus propios centroides todo el tiempo.

El siguiente fragmento código ilustra cómo el fenómeno anterior a veces puede conducir a clusters muy desequilibrados, dependiendo de la inicialización aleatoria:


In [ ]:
for seed in range(5):
    kmeans = KMeans(
        n_clusters=true_k,
        max_iter=100,
        n_init=1,
        random_state=seed,
    ).fit(vectors['tf_idf'])
    cluster_ids, cluster_sizes = np.unique(kmeans.labels_, return_counts=True)
    print(f"Number of elements asigned to each cluster: {cluster_sizes}")
print()
print(
    "True number of documents in each category according to the class labels: "
    f"{category_sizes}"
)

Vemos como cada categoría tiene entre 600 y 1000 textos, y sin embargo es frecuente encontrar clúster con entre 10 y 50 elementos. Para intentar regular esto, vamos a utilizar el parámetro `n_init`.

In [ ]:
for seed in range(5):
    kmeans = KMeans(
        n_clusters=true_k,
        max_iter=100,
        n_init=20,
        random_state=seed,
    ).fit(vectors['tf_idf'])
    cluster_ids, cluster_sizes = np.unique(kmeans.labels_, return_counts=True)
    print(f"Number of elements asigned to each cluster: {cluster_sizes}")
print()
print(
    "True number of documents in each category according to the class labels: "
    f"{category_sizes}"
)

Vemos que ahora el número de elementos de los diferentes grupos son mucho más estables, aunque esto tiene cierto impacto en los tiempos de ejecución.

### Definición de la clase

Por lo tanto, vamos a fijar los mismos parámetros del algoritmos de clústering para todas las funciones de pesado, y entre los parámetros vamos a utilizar un `n_init` de 20.

In [ ]:
kmeans = KMeans(
    n_clusters=true_k,
    max_iter=100,
    n_init=20,
    random_state=SEED,
)

## Evaluación de resultados

Vamos a definir cómo vamos a medir los resultados del clústering, el impacto de las diferentes funciones de pesado y com hacer seguimiento de los resultados.

Dado que disponemos de etiquetas de clase para este conjunto de datos específico, es posible utilizar métricas de evaluación que aprovechen esta información "supervisada" para cuantificar la calidad de los clusters resultantes.

Ejemplos de estas métricas son los siguientes:

- **Homogeneidad**, que cuantifica en qué medida los clusters contienen sólo miembros de una única clase.

- **Exhaustividad**, que cuantifica cuántos miembros de una clase determinada están asignados a los mismos clusters.

- La **medida V**, que es la media armónica de la exhaustividad y la homogeneidad.

- **Índice de Rand**, que mide la frecuencia con que los pares de puntos de datos se agrupan de forma coherente según el resultado del algoritmo de y la asignación de clase real.

- **Índice de Rand ajustado**, un índice de Rand ajustado al azar de forma que la asignación aleatoria de clusters tiene un índice esperado de 0,0.

Si no se conocen las etiquetas, la evaluación sólo puede realizarse con los propios resultados del modelo de agrupamiento. En ese caso, el [coeficiente de silueta](https://en.wikipedia.org/wiki/Silhouette_(clustering)) resulta muy útil.

Todas estas métricas de evaluación de la agrupación tienen un valor máximo de 1,0 (para un resultado de agrupación perfecto). Debermos tener en cuenta que las etiquetas de clase pueden no reflejar con exactitud los temas de los documentos y, por tanto, las métricas que utilizan etiquetas pueden no ser un sistema de evaluación óptimo.

Vamos a ir guardando los resultados de las evaluaciones en las siguientes variables:

In [ ]:
evaluations = []
evaluations_std = []

A continuación vamos a escribir una función que nos permita hacer una evaluación rápida del problema en términos de las métricas definidas. Para cada experimento utilizaremos por defecto 5 repeticiones del clustering.

In [ ]:
def quick_eval(km, X, n_runs=5, threshold=0.0):
    scores = defaultdict(list)
    for seed in range(n_runs):
        km.set_params(random_state=seed)
        t0 = time()
        km.fit(X)
        scores["Homogeneity"].append(metrics.homogeneity_score(labels, km.labels_))
        scores["Completeness"].append(metrics.completeness_score(labels, km.labels_))
        scores["V-measure"].append(metrics.v_measure_score(labels, km.labels_))
        scores["Adjusted Rand-Index"].append(
            metrics.adjusted_rand_score(labels, km.labels_)
        )
    avg_scores = []
    for score_name, score_values in scores.items():
        mean_score = np.mean(score_values)
        print(f"{score_name}: {mean_score:.3f}")
        avg_scores.append(
            {
                'score_name': score_name,
                'score': mean_score,
                'threshold': threshold
            }
        )
    return avg_scores

In [ ]:
_ = quick_eval(kmeans, vectors['tf_idf'])

Adicionalmente vamos a definir una función similar, pero que nos permita ir haciendo un seguimiento de los resultados para hacer una comparativa de resultados incluyendo los más relevantes.

In [ ]:
def track_experiment(km, X, name=None, n_runs=5):
    name = km.__class__.__name__ if name is None else name

    train_times = []
    scores = defaultdict(list)
    for seed in range(n_runs):
        km.set_params(random_state=seed)
        t0 = time()
        km.fit(X)
        train_times.append(time() - t0)
        scores["Homogeneity"].append(metrics.homogeneity_score(labels, km.labels_))
        scores["Completeness"].append(metrics.completeness_score(labels, km.labels_))
        scores["V-measure"].append(metrics.v_measure_score(labels, km.labels_))
        scores["Adjusted Rand-Index"].append(
            metrics.adjusted_rand_score(labels, km.labels_)
        )
        scores["Silhouette Coefficient"].append(
            metrics.silhouette_score(X, km.labels_, sample_size=2000)
        )
    train_times = np.asarray(train_times)

    print(f"clustering done in {train_times.mean():.2f} ± {train_times.std():.2f} s ")
    evaluation = {
        "estimator": name,
        "train_time": train_times.mean(),
    }
    evaluation_std = {
        "estimator": name,
        "train_time": train_times.std(),
    }
    for score_name, score_values in scores.items():
        mean_score, std_score = np.mean(score_values), np.std(score_values)
        print(f"{score_name}: {mean_score:.3f} ± {std_score:.3f}")
        evaluation[score_name] = mean_score
        evaluation_std[score_name] = std_score
    evaluations.append(evaluation)
    evaluations_std.append(evaluation_std)

Finalmente, vamos a generar una función para mostras las palabras más importantes de cada clúster.

In [ ]:
def get_top_words(centroids, vectorizer):
    order_centroids = centroids.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names_out()

    for i in range(true_k):
        print(f"Cluster {i}: ", end="")
        for ind in order_centroids[i, :10]:
            print(f"{terms[ind]} ", end="")
        print()

### Primeros resutados

Vamos a registrar las métricas de las representaciones con las tres funciones de pesado definidas, así como a mostrar las palabras más relevantes en cada caso para poder analizarlas a posteriori.

#### Binary

In [ ]:
track_experiment(kmeans, vectors['binary'], name="KMeans\non binary vectors")

In [ ]:
centroids = kmeans.cluster_centers_
get_top_words(centroids, vectorizers['binary'])

Vemos como las métricas son bastante bajas, y como al explorar las palabras más relevantes nos encontramos con una gran presencia de stopwords y otras palabras con baja carga semántica.

#### TF

In [ ]:
track_experiment(kmeans, vectors['tf'], name="KMeans\non tf vectors")

In [ ]:
centroids = kmeans.cluster_centers_
get_top_words(centroids, vectorizers['tf'])

En este caso, las métricas siguen siendo bajas, pero podemos observar como la selección de palabras relevantes es incluso menos significativa que en el caso anterior.

#### TF-IDF

In [ ]:
track_experiment(kmeans, vectors['tf_idf'], name="KMeans\non tf-idf vectors")

In [ ]:
centroids = kmeans.cluster_centers_
get_top_words(centroids, vectorizers['tf_idf'])

En el caso de `tf_idf` observamos un claro incremento en las métricas, lo que sugiere un potencial impacto positivo de los términos de `idf`, aunque la exploración de los términos más relevantes sigue mostrando que el clústering no está siendo tan efectivo como debería ser.

#### Comparativa

Como podemos observar, las métricas de la función de pesado que incluye `idf` son son considerablementes mejores. Esto, unido a la exploración que habíamos hecho de los valores de `idf` de los diferentes términos, empiezan a generarnos una intuición clara sobre el impacto que tiene en las representaciones, reduciendo el número de variables superfluas de la representación.

Prestando atención a las palabras más relevantes en cada caso, podemos ver que en todos los casos tenemos multitud de palabras con muy poca carga semántica.

Vamos a generar una función que nos ayude a visualizar los resultados.

In [ ]:
def plot_experiment(evaluations, evaluations_std, size=(6, 10)):
    pd.DataFrame(evaluations[::-1]).set_index("estimator")
    fig, ax = plt.subplots(figsize=size, sharey=True)

    df = pd.DataFrame(evaluations[::-1]).set_index("estimator")
    df_std = pd.DataFrame(evaluations_std[::-1]).set_index("estimator")

    df.drop(
        ["train_time"],
        axis="columns",
    ).plot.barh(ax=ax, xerr=df_std)
    ax.set_xlabel("Clustering scores")
    ax.set_ylabel("")

In [ ]:
plot_experiment(evaluations, evaluations_std, size=(8, 8))

Como vemos, las métricas de las funciones de pesado `tf`y `binary`, indican que está siendo muy complicado llevar a cabo una separación clara de los clusters. De hecho, el valor tan elevado de Homogeneidad que presentan los resultados de la función `tf` indican que probablemente exista un gran clúster que contiene muchos de los textos. 

Pare entender esto en detalle, y por qué la homogeneidad es alta, vamos a llevar a cabo un pequeño test. Vamos a ajustar de nuevo un conjunto de clusters con la función de de pesado `tf` por ejemplo, y veremos cuántas etiquetas se asignan a cada categoría.

In [ ]:
test_kmeans = KMeans(
    n_clusters=true_k,
    max_iter=100,
    n_init=20,
    random_state=SEED,
)

In [ ]:
_ = test_kmeans.fit_transform(vectors['tf'])

In [ ]:
Counter(test_kmeans.labels_)

Como podemos ver, casi todos los textos están dentro del mismo conjunto, y luego hay conjuntos con muy pocos elementos de una misma categorías, que elevan la homogoeneidad del mismo.


En la siguiente celda de código, y adelantándonos a resultados posteriores, podemos ver cómo con una reducción agresiva de la dimensionalidad perdemos poca información. Retomaremos esta línea más tarde. 

In [ ]:
lsa = make_pipeline(TruncatedSVD(n_components=100), Normalizer(copy=False))
t0 = time()
X_lsa = lsa.fit_transform(vectors['tf'])
explained_variance = lsa[0].explained_variance_ratio_.sum()

print(f"LSA done in {time() - t0:.3f} s")
print(f"Explained variance of the SVD step: {explained_variance * 100:.1f}%")

## Impacto de valores IDF

Vamos a ver ahora el impacto de los valores de `idf` a la hora de seleccionar palabras más relevantes. Para ello, vamos a estudiar qué ocurre si eliminamos algunos términos en función de sus pesos de IDF.


In [ ]:
idf_values = dict(sorted(idf_values.items(), key=lambda item: item[1], reverse=False))
len(idf_values.keys())

### Restricción de valores bajos de idf

Cómo ya habíamso visto, las palabras con valores bajso son extremadamente comunes y no sirven en general para poder diferenciar clústers ni aportan en la definición de un modelo de espacio vectorial.

Vamos a ver cómo impacta la eliminación de las palabras de peso de idf bajo. Vamos a probar con diferentes límites y extraeremos las métricas de los clusters asociados.


In [ ]:
vocab_sizes = []
experiment_scores = []
for threshold in np.arange(1,7,0.5).tolist():
    vocabulary = [word for word, idf_value in idf_values.items() if idf_value > threshold]
    print(f"\n Threshold: {threshold}, vocab size {len(vocabulary)} \n", "-"*10)
    vocab_sizes.append(
        {
            'threshold': threshold,
            'vocab_size': len(vocabulary)
        }
    )
    tf_idf_vectorizer = TfidfVectorizer(
        max_df=MAX_DF,
        min_df=MIN_DF,
        vocabulary=vocabulary,
        use_idf=True, #if False => tf_vectorize
        smooth_idf=1
    )
    X_tfidf = tf_idf_vectorizer.fit_transform(dataset.data)
    scores = quick_eval(kmeans, X_tfidf, threshold=threshold)
    experiment_scores += scores

Vamos a visualizar los resultados, para intentar encontrar un punto de corte óptimo.

In [ ]:
experiment_scores = pd.DataFrame(experiment_scores)

En la siguiente gráfica podemos visualizar los resultados del experimento previo, y ver cómo varían las métricas del clústering, en función del filtrado que se hace en función de los pesos de `idf`:

In [ ]:
sns.lineplot(data=experiment_scores, x="threshold", y="score", hue="score_name")

Cómo podemos observar, en valores cercanos a 3.5 se alcanza el máximo de las métricas exploradas. Veamos también que reducción en el vocabulario supone el filtrado por peso de idf con distintos límites. Es decir, el tamaño del vocabulario resultante tras limitarlo en función de los distintos valores de `idf`:

In [ ]:
vocab_sizes = pd.DataFrame(vocab_sizes)
ax = sns.lineplot(data=vocab_sizes, x="threshold", y="vocab_size")
ax.set(ylim=(2_000,6_000))

### Restricción de valores altos de idf

Ya hemos observado el impacto de aplicar el pesado `idf` para encontrar las palabras más relevantes a través de la penaliación o eliminación de valores bajos. A continuación veremos si ocurre algo similar al hacer un filtrado de los valores más altos, dado que en ocasiones, en los valores más altos es común encontrar algunos tokens o dígitos que a priori no deberían aportar (debidos principalmente a impurezas en el dataset). En todo caso, no debería ser un recorde demasiado grande si los textos tienen cierto nivel de limpieza.

In [ ]:
vocab_sizes = []
experiment_scores = []
for threshold in np.arange(7.5,5,-0.5).tolist():
    vocabulary = [word for word, idf_value in idf_values.items() if 3.5 < idf_value < threshold]
    print(f"\n Threshold: {threshold}, vocab size {len(vocabulary)} \n", "-"*10)
    vocab_sizes.append(
        {
            'threshold': threshold,
            'vocab_size': len(vocabulary)
        }
    )
    tf_idf_vectorizer = TfidfVectorizer(
        max_df=MAX_DF,
        min_df=MIN_DF,
        vocabulary=vocabulary,
        #norm=None,
        use_idf=True, #if False => tf_vectorize
        smooth_idf=1
    )
    X_tfidf = tf_idf_vectorizer.fit_transform(dataset.data)
    scores = quick_eval(kmeans, X_tfidf, threshold=threshold)
    experiment_scores += scores

In [ ]:
experiment_scores = pd.DataFrame(experiment_scores)

In [ ]:
sns.lineplot(data=experiment_scores, x="threshold", y="score", hue="score_name")

Como podemos observar, salvo para el índice Rand ajustado donde se puede apreciar una mejora marginal al llegar al bajar 7.0, el resto de métricas empeoran tan pronto se reduce el vocabulario.

Por tanto, parece que no deberíamos utilizar este filtrado para niveles altos. En todo caso, si ocurriese este fenómeno debido a impurezas en el dataset, la vía para solucionarlo sería la eliminación de las mismas.

### Restricción óptima

A continuación vamos a ver qué ocurre cuando aplicamos esta restricción cuando se usa cualquiera de las tres funciones de pesado, a través de una restricción de palabras basada en `idf`.

Fijaremos el límite en un peso de 3.5, por ser un valor óptimo *como* anteriormente.




In [ ]:
threshold = 3.5

In [ ]:
vocabulary = [word for word, idf_value in idf_values.items() if idf_value > threshold]

In [ ]:
tf_vectorizer = CountVectorizer(
    max_df=MAX_DF,
    min_df=MIN_DF,
    max_features=MAX_FEATURES,
    binary=False,
    vocabulary=vocabulary
)
tf_idf_vectorizer = TfidfVectorizer(
    max_df=MAX_DF,
    min_df=MIN_DF,
    max_features=MAX_FEATURES,
    use_idf=True, #if False => tf_vectorize
    smooth_idf=1,
    vocabulary=vocabulary
)
binary_vectorizer = CountVectorizer(
    max_df=MAX_DF,
    min_df=MIN_DF,
    max_features=MAX_FEATURES,
    binary=True,
    vocabulary=vocabulary
)

In [ ]:
vectorizers = {
    'tf': tf_vectorizer,
    'tf_idf': tf_idf_vectorizer,
    'binary': binary_vectorizer
}

In [ ]:
vectors = {}
for vectorizer_name, vectorizer in vectorizers.items():
    print(f'fitting {vectorizer_name} vectorizer ...')
    vectors[vectorizer_name] = vectorizer.fit_transform(dataset.data)

Podemos observar que el número de características de cada vectorización es común para todas las funciones de pesado.

In [ ]:
for vectorizer_name, X in vectors.items():
    print(f"{vectorizer_name}: n_samples: {X.shape[0]}, n_features: {X.shape[1]}")

También podemos comprobar como la densidad de valores en las matrices de las vectorizaciones se ha reducido, dado que eliminamos palabras muy presentes en un gran número de documentos.

In [ ]:
for vectorizer_name, X in vectors.items():
    print(f"{vectorizer_name}: {X.nnz / np.prod(X.shape):.3f}")

Revisemos a continuación los resultados obtenidos en cada caso particular.

#### Binary

In [ ]:
track_experiment(kmeans, vectors['binary'], name="KMeans\non binary vectors \nand idf restriction")

In [ ]:
centroids = kmeans.cluster_centers_
get_top_words(centroids, vectorizers['binary'])

Podemos ver una leve mejora en las métricas respecto a las pruebas iniciales, y cómo en algunas agrupaciones las palabras seleccionadas comienzan a tener más sentido, aunque continuamos encontrando palabras que no guardan una relación clara con el resto de términos. Por ejemplo, para el clúster de religión encontramos:

```
god believe while must fact before
```


#### TF

In [ ]:
track_experiment(kmeans, vectors['tf'], name="KMeans\non tf vectors \nand idf restriction")

In [ ]:
centroids = kmeans.cluster_centers_
get_top_words(centroids, vectorizers['tf'])

En este caso, encontramos un mayor incremento en las métricas, y agrupaciones que tienen algo más de sentido que con la función de pesado `binary`. Por ejemplo:

```
lord god christ father unto him son ps jesus said
```

#### TF-IDF

In [ ]:
track_experiment(kmeans, vectors['tf_idf'], name="KMeans\non tf-idf vectors \nand idf restriction")

In [ ]:
centroids = kmeans.cluster_centers_
get_top_words(centroids, vectorizers['tf_idf'])

Por último, en este caso la mejora ha sido considerable, y la mayoría de las palabras observadas guardan una clara relación entre sí, y definen bien el topic:

```
god jesus bible believe christian religion him christians belief faith
space nasa shuttle moon orbit launch mission earth program

```

#### Comparativa

Como vemos, en los tres casos las palabras más relevantes ya contienen muchas menos stopwords, y vemos que en los casos de tf, y sobre todo de idf, las palabras más relevantes de cada clúster comienzan a tener más relación semántica.

Revisemos a continuación las métricas obtenidas:

In [ ]:
plot_experiment(evaluations, evaluations_std, size=(8, 12))

Como habíamos observado, hay mejora en las métricas. El impacto de esta restricción tiene un gran impacto para el pesado `tf_idf`, mientras que en los otros dos casos el impacto es algo menor (aunque hay mejora), y desigual entre las distintas métricas.

Por el momento, es claro que el pesado `idf` tiene un claro impacto en la representación de los textos y cómo podemos condensar la información.

A continuación vamos a ver qué ocurre al aplicar LSI para reducir dimensionalidad.

## Aplicación de LSI

Dado que la reducción aplicada con el pesado `idf` ha tenido un impacto positivo en las métricas, vamos a ver qué ocurre al utilizar SDV para reducir la dimensionalidad, para las tres funciones de pesado.

#### Binary

In [ ]:
lsa = make_pipeline(TruncatedSVD(n_components=100), Normalizer(copy=False))
t0 = time()
X_lsa = lsa.fit_transform(vectors['binary'])
explained_variance = lsa[0].explained_variance_ratio_.sum()

print(f"LSA for binary vectorization done in {time() - t0:.3f} s")
print(f"Explained variance of the SVD step: {explained_variance * 100:.1f}%")

In [ ]:
track_experiment(kmeans, X_lsa, name="KMeans\nwith LSA on binary vectors")

In [ ]:
centroids = lsa[0].inverse_transform(kmeans.cluster_centers_)
get_top_words(centroids, vectorizers['binary'])

Tras aplicar la reducción de la dimensionalidad podemos comprobar como en el caso de esta función de pesado existe una gran mejora. Esto apunta a que la eliminación de información no relevante consigue reducir el ruido y mejorar los resultados del algoritmo de agrupamiento.

#### TF

In [ ]:
lsa = make_pipeline(TruncatedSVD(n_components=100), Normalizer(copy=False))
t0 = time()
X_lsa = lsa.fit_transform(vectors['tf_idf'])
explained_variance = lsa[0].explained_variance_ratio_.sum()

print(f"LSA for tf vectorization done in {time() - t0:.3f} s")
print(f"Explained variance of the SVD step: {explained_variance * 100:.1f}%")

In [ ]:
track_experiment(kmeans, X_lsa, name="KMeans\nwith LSA on tf vectors")

In [ ]:
centroids = lsa[0].inverse_transform(kmeans.cluster_centers_)
get_top_words(centroids, vectorizers['tf'])

Al igual que en el caso `binary`, la mejora es importante en este caso también. Veamos qué ocurre en el caso de `tf_idf`.

#### TF-IDF

In [ ]:
lsa = make_pipeline(TruncatedSVD(n_components=100), Normalizer(copy=False))
t0 = time()
X_lsa = lsa.fit_transform(vectors['tf'])
explained_variance = lsa[0].explained_variance_ratio_.sum()

print(f"LSA for tf-idf vectorization done in {time() - t0:.3f} s")
print(f"Explained variance of the SVD step: {explained_variance * 100:.1f}%")

In [ ]:
track_experiment(kmeans, X_lsa, name="KMeans\nwith LSA on tf-idf vectors")

In [ ]:
centroids = lsa[0].inverse_transform(kmeans.cluster_centers_)
get_top_words(centroids, vectorizers['tf_idf'])

Es interesante ver cómo la varianza explicada en este caso es mucho mayor que en los casos anteriores, y que a pesar de ser buenas, las métricas en este caso se han reducido levemente al aplicar la reducción de dimensionalidad.

Esto indica que el efecto del factor de `idf` y la reducción de dimensionalidad tienen un impacto similar, y que la combinación de ambas es parcialente redundante. La reducción de dimensionalidad en este caso ayuda a filtrar mucho del ruido que se provoca en este tipo de representaciones de alta dimensionalidad.

#### Comparativa

Finalmente vamos a visualizar todos los resultados obtenidos hasta el momento, de manera que podamos tener una comparativa clara de todas las técnicas aplicadas.

In [ ]:
plot_experiment(evaluations, evaluations_std, size=(8, 16))

Como podemos ver, que según hemos aplicado refinamientos sobre las técnicas iniciales las métricas no únicamente han mejorado, si no que se han estabilidado y se han igualado los resultados de las distintas funciones de pesado. 

## Conclusiones

En este ejercicio hemos observado el impacto positivo que introduce el pesado basado en la función de idf, que ayuda claramente a ponderar los términos más relevantes.

Sin embargo, un filtrado de vocabulario basado en en idf, pero aplicado a funciones de pesado que no incluyen esta técnica no se ha demos trado tan eficaz. Es interesante ver como en los casos de las funciones de pesado `tf` y `binary` podemos igualar lso resultados aplicando una reducción de dimensionalidad basada en LSI.

Podemos concluir que con estas funciones de pesado, donde se obtienen representaicones de alta dimensionalidad, es crucial introducir una técnica que ayude a filtrar o ponderar la información más relevante. Apliquemos una u otra, dada la alta presencia de información irrelevante, obtendremos resultados similares, y combinar una con otra no representa una gran mejoría respecto a utilizar únicamente una de ellas, dado que ambas tienen el mismo objetivo.